
*   Computing Platforms: Set up the Workspace for Machine Learning Projects.  https://ms.pubpub.org/pub/computing
*  Machine Learning for Predictions. https://ms.pubpub.org/pub/ml-prediction
* Machine Learning Packages: https://scikit-learn.org/stable/


# Part I: Import and Inspect Data

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/yutongquan/STATS201_Problem_Set_2/main/Data/queried_data/currency_daily_BTC_USD.csv')
df.head()

,Date,Open,High,Low,Close,Volume,Market Cap
0,2022/11/27,16457.61,16503.91,16446.59,16489.81,11754.33237,11754.33237
1,2022/11/26,16521.35,16701.99,16385.00,16458.57,181804.81670,181804.81670
2,2022/11/25,16599.55,16666.00,16342.81,16522.14,182089.49530,182089.49530
3,2022/11/24,16603.11,16812.63,16458.05,16598.95,206565.92350,206565.92350
4,2022/11/23,16227.96,16706.00,16160.20,16603.11,264927.70410,264927.70410


In [3]:
df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Market Cap'], dtype='object')

In [4]:
df.dtypes

Date           object
Open          float64
High          float64
Low           float64
Close         float64
Volume        float64
Market Cap    float64
dtype: object

# Part II: Prepare the Y varible for Regression

## 2.1. Write functions to calculte the Y variable for Regression 

*(skip the step if the Y variable already exists)*

In [5]:
df['Daily Return'] = df['Close'].pct_change()
df.head()

,Date,Open,High,Low,Close,Volume,Market Cap,Daily Return
0,2022/11/27,16457.61,16503.91,16446.59,16489.81,11754.33237,11754.33237,NaN
1,2022/11/26,16521.35,16701.99,16385.00,16458.57,181804.81670,181804.81670,-0.001895
2,2022/11/25,16599.55,16666.00,16342.81,16522.14,182089.49530,182089.49530,0.003862
3,2022/11/24,16603.11,16812.63,16458.05,16598.95,206565.92350,206565.92350,0.004649
4,2022/11/23,16227.96,16706.00,16160.20,16603.11,264927.70410,264927.70410,0.000251


## 2.2. Make Sure that the Data Type of Y is "numeric"

In [6]:
df['Daily Return']=pd.to_numeric(df['Daily Return'])
df.dtypes

Date             object
Open            float64
High            float64
Low             float64
Close           float64
Volume          float64
Market Cap      float64
Daily Return    float64
dtype: object

# Part III: Prepare the Y variable for Classification

reference:

https://datatofish.com/if-condition-in-pandas-dataframe/ *italicized text*

In [7]:
#@title Define the Congestion Threshold
cut = 0 #@param {type:"number"}


## 3.1. Method 1: If function

In [8]:
df['Positive'] = df['Daily Return'] >= cut
df.head()

,Date,Open,High,Low,Close,Volume,Market Cap,Daily Return,Positive
0,2022/11/27,16457.61,16503.91,16446.59,16489.81,11754.33237,11754.33237,NaN,False
1,2022/11/26,16521.35,16701.99,16385.00,16458.57,181804.81670,181804.81670,-0.001895,False
2,2022/11/25,16599.55,16666.00,16342.81,16522.14,182089.49530,182089.49530,0.003862,True
3,2022/11/24,16603.11,16812.63,16458.05,16598.95,206565.92350,206565.92350,0.004649,True
4,2022/11/23,16227.96,16706.00,16160.20,16603.11,264927.70410,264927.70410,0.000251,True


In [9]:
df.loc[(df['Daily Return'] >= cut), 'Positive'] = 1
df.loc[(df['Daily Return'] <cut), 'Positive'] = 0
df.head()

,Date,Open,High,Low,Close,Volume,Market Cap,Daily Return,Positive
0,2022/11/27,16457.61,16503.91,16446.59,16489.81,11754.33237,11754.33237,NaN,False
1,2022/11/26,16521.35,16701.99,16385.00,16458.57,181804.81670,181804.81670,-0.001895,0
2,2022/11/25,16599.55,16666.00,16342.81,16522.14,182089.49530,182089.49530,0.003862,1
3,2022/11/24,16603.11,16812.63,16458.05,16598.95,206565.92350,206565.92350,0.004649,1
4,2022/11/23,16227.96,16706.00,16160.20,16603.11,264927.70410,264927.70410,0.000251,1


## 3.2. Method 2: Lambda function

notes: the best method that I suggest

In [10]:
df['Positive'] = df['Daily Return'].apply(lambda x: 1 if x>= cut else 0)
df.head()

,Date,Open,High,Low,Close,Volume,Market Cap,Daily Return,Positive
0,2022/11/27,16457.61,16503.91,16446.59,16489.81,11754.33237,11754.33237,NaN,0
1,2022/11/26,16521.35,16701.99,16385.00,16458.57,181804.81670,181804.81670,-0.001895,0
2,2022/11/25,16599.55,16666.00,16342.81,16522.14,182089.49530,182089.49530,0.003862,1
3,2022/11/24,16603.11,16812.63,16458.05,16598.95,206565.92350,206565.92350,0.004649,1
4,2022/11/23,16227.96,16706.00,16160.20,16603.11,264927.70410,264927.70410,0.000251,1


# Part IV: Create the X variables

## 4.1. Shift the Y to get past values

reference:
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.shift.html

In [11]:
# generate a new variable as the previous 1 observable of your Y variable for regression
df['Daily Return_Past'] =df['Daily Return'].shift(1)
df.head()

,Date,Open,High,Low,Close,Volume,Market Cap,Daily Return,Positive,Daily Return_Past
0,2022/11/27,16457.61,16503.91,16446.59,16489.81,11754.33237,11754.33237,NaN,0,NaN
1,2022/11/26,16521.35,16701.99,16385.00,16458.57,181804.81670,181804.81670,-0.001895,0,NaN
2,2022/11/25,16599.55,16666.00,16342.81,16522.14,182089.49530,182089.49530,0.003862,1,-0.001895
3,2022/11/24,16603.11,16812.63,16458.05,16598.95,206565.92350,206565.92350,0.004649,1,0.003862
4,2022/11/23,16227.96,16706.00,16160.20,16603.11,264927.70410,264927.70410,0.000251,1,0.004649


## 4.2. Calculate the Moving Averages

references: 

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html

https://towardsdatascience.com/moving-averages-in-python-16170e20f6c

In [12]:
#@title Define the Window
window = 10 #@param {type:"number"}


In [13]:
df['Daily Return_Past_ma10']=df['Daily Return_Past'].rolling(window=window,min_periods=1).mean()
df.head(20)

,Date,Open,High,Low,Close,Volume,Market Cap,Daily Return,Positive,Daily Return_Past,Daily Return_Past_ma10
0,2022/11/27,16457.61,16503.91,16446.59,16489.81,11754.33237,11754.33237,NaN,0,NaN,NaN
1,2022/11/26,16521.35,16701.99,16385.00,16458.57,181804.81670,181804.81670,-0.001895,0,NaN,NaN
2,2022/11/25,16599.55,16666.00,16342.81,16522.14,182089.49530,182089.49530,0.003862,1,-0.001895,-0.001895
3,2022/11/24,16603.11,16812.63,16458.05,16598.95,206565.92350,206565.92350,0.004649,1,0.003862,0.000984
4,2022/11/23,16227.96,16706.00,16160.20,16603.11,264927.70410,264927.70410,0.000251,1,0.004649,0.002206
5,2022/11/22,15781.29,16315.00,15616.63,16226.94,239548.06620,239548.06620,-0.022657,0,0.000251,0.001717
6,2022/11/21,16279.50,16319.00,15476.00,15781.29,324096.99780,324096.99780,-0.027464,0,-0.022657,-0.003158
7,2022/11/20,16700.68,16753.33,16180.00,16280.23,154842.13480,154842.13480,0.031616,1,-0.027464,-0.007209
8,2022/11/19,16699.43,16822.41,16553.53,16700.68,104963.15560,104963.15560,0.025826,1,0.031616,-0.001662
9,2022/11/18,16692.56,17011.00,16546.04,16700.45,214224.18180,214224.18180,-0.000014,0,0.025826,0.001774


# Part V Train and Test Split

*reference*:

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html

In [14]:
from sklearn.model_selection import TimeSeriesSplit
tss = TimeSeriesSplit()
print(tss)

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None)


In [15]:
# change the train and test split parameters 
tss = TimeSeriesSplit(gap=0, max_train_size=None, n_splits=2, test_size=None)

In [16]:
for train_idx, test_idx in tss.split(df):
    print("TRAIN:", train_idx, "TEST:", test_idx)

TRAIN: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244

In [17]:
train_idx

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [18]:
test_idx

array([667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679,
       680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692,
       693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705,
       706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718,
       719, 720, 721, 722, 723, 724, 725, 726, 727, 728, 729, 730, 731,
       732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744,
       745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755, 756, 757,
       758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770,
       771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783,
       784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796,
       797, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809,
       810, 811, 812, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822,
       823, 824, 825, 826, 827, 828, 829, 830, 831, 832, 833, 834, 835,
       836, 837, 838, 839, 840, 841, 842, 843, 844, 845, 846, 84

In [19]:
train_df = df.filter(items=train_idx, axis=0)
test_df =  df.filter(items=test_idx, axis=0)

In [20]:
train_df.head()

,Date,Open,High,Low,Close,Volume,Market Cap,Daily Return,Positive,Daily Return_Past,Daily Return_Past_ma10
0,2022/11/27,16457.61,16503.91,16446.59,16489.81,11754.33237,11754.33237,NaN,0,NaN,NaN
1,2022/11/26,16521.35,16701.99,16385.00,16458.57,181804.81670,181804.81670,-0.001895,0,NaN,NaN
2,2022/11/25,16599.55,16666.00,16342.81,16522.14,182089.49530,182089.49530,0.003862,1,-0.001895,-0.001895
3,2022/11/24,16603.11,16812.63,16458.05,16598.95,206565.92350,206565.92350,0.004649,1,0.003862,0.000984
4,2022/11/23,16227.96,16706.00,16160.20,16603.11,264927.70410,264927.70410,0.000251,1,0.004649,0.002206


In [21]:
test_df.head()

,Date,Open,High,Low,Close,Volume,Market Cap,Daily Return,Positive,Daily Return_Past,Daily Return_Past_ma10
667,2021/1/29,33368.18,38531.90,31915.40,34252.20,231827.00560,231827.00560,-0.000312,0,0.035352,-0.028359
668,2021/1/28,30362.19,33783.98,29842.10,33364.86,92621.14562,92621.14562,-0.025906,0,-0.000312,-0.028292
669,2021/1/27,32464.01,32557.29,29241.72,30366.15,95911.96171,95911.96171,-0.089876,0,-0.025906,-0.014539
670,2021/1/26,32254.19,32921.88,30837.37,32467.77,84972.20691,84972.20691,0.069209,1,-0.089876,-0.024535
671,2021/1/25,32259.45,34875.00,31910.00,32254.20,88499.22692,88499.22692,-0.006578,0,0.069209,-0.015326


# Part VI Prepare the Train and Test Data for Classification and Regression

## 6.1. Classification

### 6.1.1 Define the columns (Y, X) for Classification 

In [22]:
cols_C = ['Positive','Daily Return_Past_ma10']

### 6.1.2 Define the Data Frame of Train and Test Data for Classification

In [23]:
df_C_train = train_df[cols_C]
df_C_test = test_df[cols_C]

### 6.1.3 Export the Train and Test Data for Classification

In [24]:
df_C_train.head()

,Positive,Daily Return_Past_ma10
0,0,NaN
1,0,NaN
2,1,-0.001895
3,1,0.000984
4,1,0.002206


In [25]:
df_C_train.to_csv('Classification_Train.csv')

In [26]:
df_C_test.head()

,Positive,Daily Return_Past_ma10
667,0,-0.028359
668,0,-0.028292
669,0,-0.014539
670,1,-0.024535
671,0,-0.015326


In [27]:
df_C_test.to_csv('Classification_Test.csv')

## 6.2 Regression

### 6.2.1. Define the columns (Y, X) for Regression

In [28]:
cols_R = ['Daily Return','Daily Return_Past_ma10']

### 6.2.2. Define the Data Frame of Train and Test Data for Regression

In [29]:
df_R_train = train_df[cols_R]
df_R_test = test_df[cols_R]

### 6.2.3. Export the Train and Test Data for Regression

In [30]:
df_R_train.head()

,Daily Return,Daily Return_Past_ma10
0,NaN,NaN
1,-0.001895,NaN
2,0.003862,-0.001895
3,0.004649,0.000984
4,0.000251,0.002206


In [31]:
df_R_train.to_csv('Regression_Train.csv')

In [32]:
df_R_test.head()

,Daily Return,Daily Return_Past_ma10
667,-0.000312,-0.028359
668,-0.025906,-0.028292
669,-0.089876,-0.014539
670,0.069209,-0.024535
671,-0.006578,-0.015326


In [33]:
df_R_test.to_csv('Regression_Test.csv')